# ZHUNT

In [13]:
zhunt_impl = r"""
/*
Z-HUNT-2 computer program, Sept.19, 1990
*/

/*
serialized i/o to allow to run against large datasets 
Updated i/o permissions to match posix. campt 1/10/2000
*/

/*
Turbo C compiler Ver. 1.5
Compact model, 8086/80286 instruction set, math emulator/8087/80287
Speed option, unsigned char
Run on IBM PC XT/AT or compatibles
*/

/*
Written by Ping-jung Chou, under the instruction of Pui S. Ho, according to the
paper "A computer aided thermodynamic approach for predicting the formation of
Z-DNA in naturally occurring sequences",
The EMBO Journal, Vol.5, No.10, pp2737-2744, 1986
With 0.22 kcal/mol/dinuc for mCG (Zacharias et al, Biochemistry, 1988, 2970)
*/


#include <stdlib.h>
#include <stdio.h>
#include <math.h>
#include <string.h>
#include <time.h>

/* creates a temporary file and mmaps the sequence into it */
#ifdef USE_MMAP
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#endif


double linear_search( double x1, double x2, double tole, double (*func)() );
double delta_linking( double dl );
double delta_linking_slope( double dl );
void   show_probability( unsigned seqlength, float *dl, float *slope, float *probability, char *sequence, char **antisyn, char *filename, int low, int high);

double linear_search( double x1, double x2, double tole, double (*func)() )
{
  double f, fmid, dx, xmid, x;

  f = func( x1 );
  fmid = func( x2 );
  if( f * fmid >= 0.0 )
    return x2;
  x = (f < 0.0) ? (dx = x2 - x1, x1) : (dx = x1 - x2, x2);
  do
    {
      dx *= 0.5;
      xmid = x + dx;
      fmid = func( xmid );
      if( fmid <= 0.0 )
        x = xmid;
    }  while( fabs( dx ) > tole );
  return x;
}





int    terms;
double *bztwist, *logcoef, *exponent;
double _k_rt = -0.2521201;              /* -1100/4363 */
double sigma = 16.94800353;             /* 10/RT */
double deltatwist;
double explimit = -600.0;
int low, high ;


double delta_linking( double dl )
{
  double sump, sumq, z, expmini;
  int    i;

  expmini = 0.0;
  for( i=0; i<terms; i++ )
    {
      z = dl - bztwist[i];
      exponent[i] = z = logcoef[i] + _k_rt * z * z;
      if( z < expmini )
        expmini = z;
    }
  expmini = (expmini < explimit) ? explimit - expmini : 0.0;
  sump = sumq = 0.0;
  for( i=0; i<terms; i++ )
    {
      z = exp( exponent[i] + expmini );
      sumq += z;
      sump += bztwist[i] * z;
    }
  sumq += exp( _k_rt * dl * dl + sigma + expmini );
  return deltatwist - sump / sumq;
}





double delta_linking_slope( double dl )
{
  double sump, sump1, sumq, sumq1, x, y, z, expmini;
  int    i;

  expmini = 0.0;
  for( i=0; i<terms; i++ )
    {
      z = dl - bztwist[i];
      exponent[i] = z = logcoef[i] + _k_rt * z * z;
      if( z < expmini )
        expmini = z;
    }
  expmini = (expmini < explimit) ? explimit - expmini : 0.0;
  sump = sump1 = sumq = sumq1 = 0.0;
  x = 2.0 * _k_rt;
  for( i=0; i<terms; i++ )
    {
      z = dl - bztwist[i];
      y = exp( exponent[i] + expmini );
      sumq += y;
      sump += bztwist[i] * y;
      y *= z * x;
      sumq1 += y;
      sump1 += bztwist[i] * y;
    }
  y = exp( _k_rt * dl * dl + sigma + expmini );
  sumq += y;
  sumq1 += x * dl * y;
  return (sump1 - sump * sumq1 / sumq) / sumq;
}                                       /* slope at delta linking = dl */





                                        /* Delta BZ Energy of Dinucleotide */
double dbzed[4][17] = {
/* AS-AS */
{ 4.40, 6.20, 3.40, 5.20, 2.50, 4.40, 1.40, 3.30, 3.30, 5.20, 2.40, 4.20, 1.40, 3.40, 0.66, 2.40 ,4.26},
/* SA-SA */
{ 4.40, 2.50, 3.30, 1.40, 6.20, 4.40, 5.20, 3.40, 3.40, 1.40, 2.40, 0.66, 5.20, 3.30, 4.20, 2.40 ,4.26},
/* AS-SA */
{ 6.20, 6.20, 5.20, 5.20, 6.20, 6.20, 5.20, 5.20, 5.20, 5.20, 4.00, 4.00, 5.20, 5.20, 4.00, 4.00 ,4.26 },
/* SA-AS */
{ 6.20, 6.20, 5.20, 5.20, 6.20, 6.20, 5.20, 5.20, 5.20, 5.20, 4.00, 4.00, 5.20, 5.20, 4.00, 4.00 ,4.26 }
                      };

double expdbzed[4][17];                 /* exp(-dbzed/rt) */
int    *bzindex;                        /* dinucleotides */



int      user_regret( void );
FILE     *open_file( int mode, char *filename, char *typestr );
void     assign_bzenergy_index( int nucleotides, char seq[] );
void     best_anti_syn( int dinucleotides, float esum );
void     anti_syn_energy(int dinculeotide, int dinucleotides, float esum );
unsigned input_sequence( FILE *file, int nucleotides, int showfile );
double   assign_probability( double dl );



void assign_bzenergy_index( int nucleotides, char seq[] )
{
  int  i, j, idx;
  char c1, c2;

  i = j = 0;
  do
    {
      c1 = seq[i++];
      c2 = seq[i++];
      switch( c1 )
        {
          case 'a' : switch( c2 )
                       {
                         case 'n' : idx = 16;  break;
			 case 'a' : idx = 0;  break;
                         case 't' : idx = 1;  break;
                         case 'g' : idx = 2;  break;
                         case 'c' : idx = 3;  break;
                       }  break;
          case 't' : switch( c2 )
                       {
                         case 'n' : idx = 16;  break;
			 case 'a' : idx =  4;  break;
                         case 't' : idx =  5;  break;
                         case 'g' : idx =  6;  break;
                         case 'c' : idx =  7;  break;
                       }  break;
          case 'g' : switch( c2 )
                       {
                         case 'n' : idx = 16;  break;
			 case 'a' : idx =  8;  break;
                         case 't' : idx =  9;  break;
                         case 'g' : idx = 10;  break;
                         case 'c' : idx = 11;  break;
                       }  break;
          case 'c' : switch( c2 )
                       {
                         case 'n' : idx = 16;  break;
			 case 'a' : idx = 12;  break;
                         case 't' : idx = 13;  break;
                         case 'g' : idx = 14;  break;
                         case 'c' : idx = 15;  break;
                       }  break;
          case 'n' : switch( c2 )
                       {
                         case 'a' : idx = 16;  break;
                         case 't' : idx = 16;  break;
                         case 'g' : idx = 16;  break;
			 case 'n' : idx = 16;  break;
                         case 'c' : idx = 16;  break;
                       }  break;					   
					   
        }
      bzindex[j++] = idx;
    }  while( i < nucleotides );
}





double *bzenergy, *best_bzenergy;       /* dinucleotides */
float  best_esum;               /* assigned before call to anti_syn_energy() */
char   *best_antisyn, *antisyn;         /* nucleotides */



void best_anti_syn( int dinucleotides, float esum )
{
  int i;
  double dl, slope;

  if( esum < best_esum )
    {
      best_esum = esum;
      for( i=0; i<dinucleotides; i++ )
        best_bzenergy[i] = bzenergy[i];
      strcpy( best_antisyn, antisyn );
    }
}





void anti_syn_energy( int din, int dinucleotides, float esum )
{
  int   i, nucleotides;
  float e;

  nucleotides = 2 * din;

  antisyn[nucleotides] = 'A';
  antisyn[nucleotides+1] = 'S';
  i = (din == 0) ? 0 : ((antisyn[nucleotides-1] == 'S') ? 0 : 3);
  e = dbzed[i][bzindex[din]];
  esum += e;
  bzenergy[din] = expdbzed[i][bzindex[din]];
  if( ++din == dinucleotides )
    best_anti_syn( dinucleotides, esum );
  else
    anti_syn_energy( din, dinucleotides, esum );
  esum -= e;
  din --;

  antisyn[nucleotides] = 'S';
  antisyn[nucleotides+1] = 'A';
  i = (din == 0) ? 1 : ((antisyn[nucleotides-1] == 'A') ? 1 : 2);
  esum += dbzed[i][bzindex[din]];
  bzenergy[din] = expdbzed[i][bzindex[din]];
  if( ++din == dinucleotides )
    best_anti_syn( dinucleotides, esum );
  else
    anti_syn_energy( din, dinucleotides, esum );
}





char *tempstr, *sequence;
#ifdef USE_MMAP
int sequencefile;
#endif

int user_regret( void )
{
  char c;
  do
    {
      gets( tempstr );
      c = tempstr[0];
    }  while( c == 0 );
  return (c == '@') ? 1 : 0;
}





FILE *open_file( int mode, char *filename, char *typestr )
{
  static char *iostr[] = { "output", "input" };
  static char *rwstr[] = { "w",     "r" };
  char  *fullfile;
  
  FILE *file;
  file = NULL;
  fullfile = (char *) malloc(sizeof(char) * ( strlen(filename) + strlen(typestr) + 1) );
  strcpy(fullfile, filename);
  if(strlen(typestr) != 0)
	{
        strcat(fullfile, ".");
	strcat(fullfile, typestr);
	}
printf("opening %s\n", fullfile);
   
  file = fopen( fullfile, rwstr[mode] );
  free(fullfile);
  return file;
}





unsigned input_sequence( FILE *file, int nucleotides, int showfile )
{
unsigned length, i, j;
char     c;
#ifdef USE_MMAP
FILE *OUTPUT;
#endif

printf("inputting sequence\n");

length = 0;                           /* count how many bases */
while( j=0, fgets( tempstr, 128, file ) != NULL )
	{
    	while( (c=tempstr[j++]) != 0 )
		{
      		if( c == 'a'  ||  c == 't'  ||  c == 'g'  ||  c == 'c'||  c == 'n'  || c == 'A'  ||  c == 'T'  ||  c == 'G'  ||  c == 'C'||  c == 'N'  )
			{
        		length ++;
			}
		}
	}

#ifndef USE_MMAP
sequence = (char *) malloc( length+nucleotides );
#else
sequence = (char *) malloc( nucleotides );
OUTPUT = (FILE *) fopen ("/usr/local/apache/htdocs/zhunt/temp", "w");
#endif

rewind( file );

if( showfile )
	{
    	printf( "\n" );
	}
i = 0;
while( j=0, fgets( tempstr, 128, file ) != NULL )
	{
    	while( (c=tempstr[j++]) != 0 )
		{
      		if( c == 'a'  ||  c == 't'  ||  c == 'g'  ||  c == 'c' || c == 'n' ||  c == 'A' ||  c == 'T'  ||  c == 'G' ||  c == 'C' ||  c == 'N')
        		{
				if( c == 'N')
				{
				c = 'n';
				}	
          		if( c == 'A')
				{
				c = 'a';
				}
          		if( c == 'T')
				{
				c = 't';
				}
          		if( c == 'G')
				{
				c = 'g';
				};
          		if( c == 'C')
				{
				c = 'c';
				}
#ifndef USE_MMAP
          		sequence[i++] = c;
#else
			fprintf(OUTPUT,"%c",c);
			if(i < nucleotides)
				{
          			sequence[i++] = c;
				}
#endif
        		}
		}
	}
 for( j=0; j<nucleotides; j++ )        /* assume circular nucleotides */
	{
#ifndef USE_MMAP
    	sequence[i++] = sequence[j];
#else
	fprintf(OUTPUT,"%c",sequence[j]);
#endif
	}
#ifdef USE_MMAP
	close(OUTPUT);	
	sequencefile = open ("/usr/local/apache/htdocs/zhunt/temp", O_RDWR, NULL);
	free(sequence);
	sequence = (char *) mmap(0,length, PROT_READ | PROT_WRITE,MAP_SHARED,sequencefile,0);
#endif

return length;
}





/* calculate the probability of the value 'dl' in a Gaussian distribution */
/* from "Data Reduction and Error Analysis for the Physical Science" */
/* Philip R. Bevington, 1969, McGraw-Hill, Inc */


double assign_probability( double dl )
{
  static double average = 29.6537135;
  static double stdv = 2.71997;
  static double _sqrt2 = 0.70710678118654752440;   /* 1/sqrt(2) */
  static double _sqrtpi = 0.564189583546;  /* 1/sqrt(pi) */

  double x, y, z, k, sum;

  z = fabs( dl - average ) / stdv;
  x = z * _sqrt2;
  y = _sqrtpi * exp( -x * x );
  z *= z;
  k = 1.0;
  sum = 0.0;
  do
    {
      sum += x;
      k += 2.0;
      x *= z / k;
    }  while( sum + x > sum );
  z = 0.5 - y * sum;                    /* probability of each tail */
  return (dl > average) ? z : 1.0/z;
}





double find_delta_linking( int dinucleotides );
void   calculate_zscore( double a, int maxdinucleotides, int min, int max, char *filename );

void   print_array( int points[], unsigned from, unsigned to, int printwidth );
void   soft_copy( int width, int widthbyte, char **screen );

void   generate_random_sequence( int dinucleotides, char seq[] );
void   initiate_random( long seed );
double uniform_random( void );
void   random_distribution( double a, int maxdinucleotides, char *filename );
void   run_distribution( double a, int maxdinucleotides, FILE *disfile );
void   print_distribution( char *disname, FILE *disfile );






double find_delta_linking( int dinucleotides )
{
  double sum;
  int    i, j;

  for( i=0; i<dinucleotides; i++ )
    bzenergy[i] = 1.0;
  for( i=0; i<dinucleotides; i++ )
    {
      sum = 0.0;
      for( j=0; j<dinucleotides-i; j++ )
        {
          bzenergy[j] *= best_bzenergy[i+j];
          sum += bzenergy[j];
        }
      logcoef[i] = log( sum );
    }
  terms = dinucleotides;
  return linear_search( 10.0, 50.0, 0.001, delta_linking );
}





void calculate_zscore( double a, int maxdinucleotides, int min, int max, char *filename )
{
  static double pideg=57.29577951;      /* 180/pi */
  char     *bestantisyn;
  FILE     *file;
  unsigned seqlength, i, j, k;
  int      fromdin, todin, din, nucleotides;
  long     begintime, endtime;
  double   dl, slope, probability, bestdl;
  float    initesum;

  fromdin = min;
  todin = max;
  printf("calculating zscore\n");

  file = open_file( 1, filename, "" );
  if( file == NULL )
	{
	printf("couldn't open %s!\n", filename);
        return;
	}
  seqlength = input_sequence( file, 2*maxdinucleotides, 0 );
  fclose( file );

  file = open_file( 0, filename, "Z-SCORE" );
  if( file == NULL )
    	{
#ifndef USE_MMAP
      	free( sequence );
#else
	munmap(sequence, seqlength);
	close(sequencefile);
#endif
      	return;
    	}

  if( todin > maxdinucleotides )
	{
    	todin = maxdinucleotides;
	}
  if( fromdin > todin )
	{
    	fromdin = todin;
	}
  nucleotides = 2 * todin;

#ifndef PROB_ONLY
  fprintf( file, "%s %u %d %d\n", filename, seqlength, fromdin, todin );
#endif

  a /= 2.0;
  initesum = 10.0 * todin;
  bestantisyn = (char *) malloc( nucleotides+1 );

  time( &begintime );
  for( i=0; i < seqlength; i++ )
    {
      assign_bzenergy_index( nucleotides, sequence+i );
      bestdl = 50.0;
      for( din=fromdin; din<=todin; din++ )
        {
          best_esum = initesum;
          deltatwist = a * (double)din;
          antisyn[2*din] = 0;
          anti_syn_energy( 0, din, 0.0 );           /* esum = 0.0 */
          dl = find_delta_linking( din );  
          if( dl < bestdl )
            {
              bestdl = dl;
              strcpy( bestantisyn, best_antisyn );
            }
        }
		j = strlen(bestantisyn);
#ifndef PROB_ONLY
      slope = atan( delta_linking_slope( bestdl ) ) * pideg;
#endif
      probability = assign_probability( bestdl );
#ifndef PROB_ONLY
      fprintf( file, "%i %i %i %7.3lf %7.3lf %le ", i+1, i+1+j, j, bestdl, slope, probability ); 
#else
      fprintf( file, " %7.3lf %le\n", bestdl, probability ); 
#endif
	for( k=0; k<j; k++ )
        fprintf( file,"%c", sequence[i+k] );
        fprintf( file, "   %s\n", bestantisyn );
        
	}
  time( &endtime );

  free( bestantisyn );
  fclose( file );
  printf( "\n run time=%ld sec\n", endtime-begintime );
#ifndef USE_MMAP
  free( sequence );
#else
  munmap(sequence,seqlength);
  close(sequencefile);
#endif
}



void analyze_zscore(  char *filename, int low, int high)
{
  float    *dl, *slope, *probability;
  unsigned seqlength, i, select;
  char     **antisyn;
  FILE     *file;
  int      fromdin, todin, nucleotides;
  

  printf("use min/max %d %d\n",low,high);
  printf("analyzing_zscore\n");

  file = open_file( 1, filename, "Z-SCORE" );
  if( file == NULL )
	{
	printf("couldn't open %s.Z-SCORE!\n",filename);
        return;
	}
  fscanf( file, "%s %u %d %d", tempstr, &seqlength, &fromdin, &todin );
  nucleotides = 2 * todin;
  dl = (float *) calloc( seqlength, sizeof(float) );
  slope = (float *) calloc( seqlength, sizeof(float) );
  probability = (float *) calloc( seqlength, sizeof(float) );
  antisyn = (char **) calloc( seqlength, sizeof(char *) );

  for( i=0; i<seqlength; i++ )
    {
      fscanf( file, "%f %f %f %s", dl+i, slope+i, probability+i, tempstr );
      antisyn[i] = strdup( tempstr );
    }
  fclose( file );

  file = open_file(1, filename , "");
  input_sequence( file, nucleotides, 0 );
  fclose( file );
/* uncomment to print probability file
 show_probability( seqlength, dl, slope, probability, sequence, antisyn,filename,low,high); 
 */
  free( dl );
  free( slope );
  free( probability );
  for( i=0; i<seqlength; i++ )
    free( antisyn[i] );
  free( antisyn );
 
#ifndef USE_MMAP
  free( sequence );
#else
  munmap(sequence,seqlength);
  close(sequencefile);
#endif
}





long RAm=714025, RAa=1366, RAc=150889, RAseed=0, RAy, RAr[97];



void initiate_random( long seed )
{
  int i;

  seed = (RAc - seed) % RAm;
  if( seed < 0 )
    seed = -seed;
  for( i=0; i<97; i++ )
    RAr[i] = seed = (RAa * seed + RAc) % RAm;
  seed = (RAa * seed + RAc) % RAm;
  RAseed = RAy = seed;
}





double uniform_random( void )
{
  int    i;

  i = (double)97.0 * RAy / (double)RAm;
  RAy = RAr[i];
  RAr[i] = RAseed = (RAa * RAseed + RAc) % RAm;
  return (double)RAy / RAm;
}





void generate_random_sequence( int dinucleotides, char seq[] )
{
  static char rs1[] = "aaaattttggggcccc";
  static char rs2[] = "atgcatgcatgcatgc";
  int i, j, r;

  for( i=j=0; i<dinucleotides; i++ )
    {
      r = uniform_random() * 17.0;
      seq[j++] = rs1[r];
      seq[j++] = rs2[r];
    }
}





void random_distribution( double a, int maxdinucleotides, char *filename )
{
  int  select;
  FILE *file;

  do
    {
      printf( "\n 0. Stop" );
      printf( "\n 1. Run distribution" );
      printf( "\n\n Select: " );
      scanf( "%d", &select );
      if( select > 0 )
        {
          file = open_file( select-1, filename, "DISTRIBUTION" );
          if( file == NULL )
            continue;
          if( select == 1 )
            run_distribution( a, maxdinucleotides, file );
         fclose( file );
        }
    }  while( select != 0 );
}





void run_distribution( double a, int maxdinucleotides, FILE *disfile )
{
  int    i, nucleotides, dinucleotides, levels;
  long   begintime, endtime, repeat, k, *distribution;
  float  initesum;
  double dl, sumdl, sumdldl, dlaverage, dlstdv, dlfrom, dlto, dlstep;

  printf( "\n Window size (dinucleotides) and sample size: " );
  if( user_regret() )
    return;
  sscanf( tempstr, "%d %ld", &dinucleotides, &repeat );
  if( dinucleotides > maxdinucleotides )
    dinucleotides = maxdinucleotides;
  nucleotides = 2 * dinucleotides;

  printf( "\n FROM, TO & STEP linking differences for distribution: " );
  if( user_regret() )
    return;
  sscanf( tempstr, "%lf %lf %lf", &dlfrom, &dlto, &dlstep );

  sequence = (char *) malloc( nucleotides );
                                   /* half of the window convert from B to Z */
  deltatwist = a * (double)dinucleotides / 2.0;
  initesum = 10.0 * dinucleotides;      /* for best_anti_syn() use */

  levels = (dlto - dlfrom) / dlstep + 1;
  distribution = (long *) calloc( levels, sizeof(long) );
  for( i=0; i<levels; i++ )
    distribution[i] = 0;
  sumdl = sumdldl = 0.0;

  time( &begintime );
  initiate_random( begintime );

  for( k=0; k<repeat; k++ )
    {
      generate_random_sequence( dinucleotides, sequence );
      assign_bzenergy_index( nucleotides, sequence );
      best_esum = initesum;
      anti_syn_energy( 0, dinucleotides, 0.0 );
      dl = find_delta_linking( dinucleotides );
      sumdl += dl;
      sumdldl += dl * dl;
      i = (dl - dlfrom) / dlstep;
      if( i < 0 )
        i = 0;
      if( i > levels - 1 )
        i = levels - 1;
      distribution[i] ++;
    }
  time( &endtime );
  dlaverage = sumdl / repeat;
  dlstdv = sqrt( (sumdldl - sumdl * dlaverage) / repeat );

  fprintf( disfile, " %d %d %ld %lf %lf %lg %lg %lg\n",
      dinucleotides, levels, repeat, dlaverage, dlstdv, dlfrom, dlto, dlstep );
  for( i=0; i<levels; i++ )
    fprintf( disfile, "%ld\n", distribution[i] );

  printf( "\n run time: %ld sec\n", endtime-begintime );
  free( distribution );
  free( sequence );
}


void show_probability( unsigned seqlength,float *dl, float *slope, float *probability, char *sequence, char **antisyn, char *filename,int low, int high)
{
  unsigned i, j, k;
  FILE     *file;
  
  printf("show_probability\n");
  printf("min/max %d %d\n",low,high);

  file = open_file( 0, filename, "probability" );
 
  if( file == NULL )
	{
	printf("couldn't open %s.probability!\n",filename);
        return;
	}
	fprintf( file, "%s %u %i %i\n", filename, seqlength, low, high );
  for( i=0; i < seqlength; i++ )
        {
		j = strlen( antisyn[i] );
		fprintf( file, " %i %i %i %7.3f %7.3f  %10.3e  ", i+1, i+1+j, j, dl[i], slope[i], probability[i] );
        
        for( k=0; k<j; k++ )
        fprintf( file, "%c", sequence[i+k] );
        fprintf( file, "   %s\n", antisyn[i] );
        }

 fclose( file );
}


int main( int argc, char *argv[])
{
    static double rt=0.59004;             /* 0.00198*298 */
    static double a=0.357, b=0.4;         /* a = 2 * (1/10.5 + 1/12) */
    double   ab;
    int      i, j, nucleotides, dinucleotides, select;
    int min, max;
    int low, high;

    if(argc < 5)
    {
        printf("usage: zhunt windowsize minsize maxsize datafile\n");
        exit(1);
    }
    tempstr = (char *) malloc( 128 );
    dinucleotides = atoi((char *)argv[1]);
    min = atoi((char *)argv[2]);
    max = atoi((char *)argv[3]);
    low = atoi((char *)argv[2]);
    high = atoi((char *)argv[3]);

    printf("dinucleotides %d\n", dinucleotides);
    printf("min/max %d %d\n", min, max);
    printf("min/max %d %d\n", low, high);
    printf("operating on %s\n", (char *)argv[4]);

    nucleotides = 2 * dinucleotides;

    antisyn = (char *) malloc( nucleotides+1 );
    best_antisyn = (char *) malloc( nucleotides+1 );

    bzindex = (int *) calloc( dinucleotides, sizeof(int) );
    bzenergy = (double *) calloc( dinucleotides, sizeof(double) );
    best_bzenergy = (double *) calloc( dinucleotides, sizeof(double) );

    bztwist = (double *) calloc( dinucleotides, sizeof(double) );
    ab = b + b;
    for( i=0; i<dinucleotides; i++ )
    {
        ab += a;
        bztwist[i] = ab;
    }

    for( i=0; i<4; i++ )
        for( j=0; j<17; j++ )
            expdbzed[i][j] = exp( -dbzed[i][j] / rt );

    logcoef = (double *) calloc( dinucleotides, sizeof(double) );
    exponent = (double *) calloc( dinucleotides, sizeof(double) );

    calculate_zscore( a, dinucleotides, min, max, (char *)argv[4] );
#ifndef PROB_ONLY
    analyze_zscore((char *)argv[4],low,high);
#endif

    free( exponent );
    free( logcoef );
    free( bztwist );
    free( best_bzenergy );
    free( bzenergy );
    free( bzindex );
    free( best_antisyn );
    free( antisyn );
    free( tempstr );
    return 0;
}

"""
with open("zhunt3-alan.c", "w", encoding="utf-8") as f:
    f.write(zhunt_impl)

In [14]:
!gcc zhunt3-alan.c -Ofast -lm -o zhunt3

clang: warning: argument '-Ofast' is deprecated; use '-O3 -ffast-math' for the same behavior, or '-O3' to enable only conforming optimizations [-Wdeprecated-ofast]
zhunt3-alan.c:52:11: warning: passing arguments to a function without a prototype is deprecated in all versions of C and is not supported in C23 [-Wdeprecated-non-prototype]
   52 |   f = func( x1 );
      |           ^
zhunt3-alan.c:53:14: warning: passing arguments to a function without a prototype is deprecated in all versions of C and is not supported in C23 [-Wdeprecated-non-prototype]
   53 |   fmid = func( x2 );
      |              ^
zhunt3-alan.c:61:18: warning: passing arguments to a function without a prototype is deprecated in all versions of C and is not supported in C23 [-Wdeprecated-non-prototype]
   61 |       fmid = func( xmid );
      |                  ^
zhunt3-alan.c:304:7: warning: 'gets' is deprecated: This function is provided for compatibility reasons only.  Due to security concerns inherent in the de

In [8]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/022/581/195/GCF_022581195.2_ilHelZeax1.1/GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz
!gunzip ./GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz
!mv GCF_022581195.2_ilHelZeax1.1_genomic.fna genomic.fna

--2025-06-18 00:23:44--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/022/581/195/GCF_022581195.2_ilHelZeax1.1/GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz
Распознаётся ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)… 130.14.250.10, 130.14.250.31, 130.14.250.13, ...
Подключение к ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 120848234 (115M) [application/x-gzip]
Сохранение в: «GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz»

GCF_022581195.2_ilH 100%[===================>] 115,25M  2,07MB/s    за 45s     

2025-06-18 00:24:29 (2,59 MB/s) - «GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz» сохранён [120848234/120848234]



In [15]:
!./zhunt3 12 8 12 genomic.fna

dinucleotides 12
min/max 8 12
min/max 8 12
operating on genomic.fna
calculating zscore
opening genomic.fna
inputting sequence
opening genomic.fna.Z-SCORE

 run time=13342 sec
use min/max 8 12
analyzing_zscore
opening genomic.fna.Z-SCORE
opening genomic.fna
inputting sequence


In [16]:
!head genomic.fna.Z-SCORE

genomic.fna 375181922 8 12
1 17 16  32.498  14.812 1.478861e-01 nccaaattacaccagn   ASASASSASASAASAS
2 18 16  30.515  17.723 3.757344e-01 ccaaattacaccagnt   ASASASASASASASAS
3 19 16  31.913  25.037 2.031080e-01 caaattacaccagntg   ASASSASAASASASAS
4 20 16  30.501  17.603 3.776943e-01 aaattacaccagntgn   ASASASASASASASAS
5 23 18  31.545  28.752 2.434462e-01 aattacaccagntgnncc   SASASAASASASASASAS
6 22 16  30.500  17.222 3.777796e-01 attacaccagntgnnc   ASASASASASASASAS
7 23 16  31.187  31.021 2.864575e-01 ttacaccagntgnncc   SASAASASASASASAS
8 24 16  30.507  17.142 3.768418e-01 tacaccagntgnnccc   ASASASASASASASAS
9 25 16  31.006  28.613 3.094738e-01 acaccagntgnnccct   SAASASASASSASASA


In [24]:
from Bio import SeqIO

In [26]:
seg_dict = {}
sum = 1
input_file = "genomic.fna"
fasta_sequence = SeqIO.parse(input_file,'fasta')

for record in SeqIO.parse(input_file, "fasta"):
    name, sequence = record.id, str(record.seq)
    seg_dict[name] = (sum, sum + len(sequence) - 1)
    sum += len(sequence)
seg_dict

{'NC_061452.1': (1, 15512169),
 'NC_061453.1': (15512170, 30573753),
 'NC_061454.1': (30573754, 45268912),
 'NC_061455.1': (45268913, 59434628),
 'NC_061456.1': (59434629, 73485891),
 'NC_061457.1': (73485892, 87509142),
 'NC_061458.1': (87509143, 101159065),
 'NC_061459.1': (101159066, 114808336),
 'NC_061460.1': (114808337, 128423600),
 'NC_061461.1': (128423601, 141681985),
 'NC_061462.1': (141681986, 154770825),
 'NC_061463.1': (154770826, 167743605),
 'NC_061464.1': (167743606, 180584909),
 'NC_061465.1': (180584910, 193342856),
 'NC_061466.1': (193342857, 205948777),
 'NC_061467.1': (205948778, 218281433),
 'NC_061468.1': (218281434, 230536315),
 'NC_061469.1': (230536316, 242781788),
 'NC_061470.1': (242781789, 254411634),
 'NC_061471.1': (254411635, 266011388),
 'NC_061472.1': (266011389, 277472905),
 'NC_061473.1': (277472906, 288343946),
 'NC_061474.1': (288343947, 298766083),
 'NC_061475.1': (298766084, 308508591),
 'NC_061476.1': (308508592, 317789078),
 'NC_061477.1': (317

In [28]:
!rm 'results_zhunt.bed'

with open('genomic.fna.Z-SCORE', 'r') as f:
    ind = -1
    while 1:
        ind += 1
        line = f.readline()
        if not line:
            break
        if ind < 1:
            continue
        start = int(line.split()[0])
        end = int(line.split()[1])
        score = float(line.split()[5])
        name_start = ''
        name_end = ''

        if score < 400:
            continue

        for key, (l, r) in seg_dict.items():
            if start >= l and start <= r:
                name_start = key
                start -= l
                if end >= l and end <= r:
                    name_end = key
                    end -= l
                break

        if name_start != name_end:
            continue

        if name_start == '':
            continue

        with open('results_zhunt.bed', 'a') as g:
            g.write(f'{name_start}\t{start}\t{end}\n')

!head 'results_zhunt.bed'

rm: results_zhunt.bed: No such file or directory
NC_061452.1	3538	3554
NC_061452.1	4526	4542
NC_061452.1	9982	9998
NC_061452.1	10589	10607
NC_061452.1	10591	10607
NC_061452.1	10597	10615
NC_061452.1	10598	10616
NC_061452.1	14209	14229
NC_061452.1	14211	14229
NC_061452.1	14212	14232
